# Ungraded Lab - Building a Doubly Linked List Class with an LLM

Welcome to the first ungraded lab of this course! In this lab you'll be working alongside an LLM to update a Linked List class to make it doubly linked. This is a good opportunity to practice your LLM prompting skills and prepare yourself for the programming assignment at the end of this course.

# Outline
- [ 1 - Introduction](#1)
  - [ 1.1 Importing necessary libraries](#1.1)
- [ 2 - The `Node` and `LinkedList` Classes to Update](#2)
- [ 3 - Test Your Classes](#3)
- [ 4 - Go Further with Your LLM Prompting Skills](#4)

<a name="1"></a>
## 1 - Introduction

**Your Task:** Below you'll find the `Node` and `LinkedList` class you saw in the lectures. Your job is to work alongside an LLM to update this class to be a doubly linked list, meaning each node has connections to both its previous and next node. Once you've done that, work with the LLM to further refine the class to account for other concerns common in software engineering like security concerns or scalability. 

**LLM Access:** You can access OpenAI's GPT-3.5 model [here](https://www.coursera.org/learn/introduction-to-generative-ai-for-software-development/ungradedLab/Vuqvf/gpt-3-5-environment), but feel free to use the LLM you want!

**Practice Prompting:** Focus on trying out the prompting skills covered in the lectures:

* **Be Specific:** In your prompts provide detail about what you're trying to accomplish and the context in which you're working. For example, it'd be totally appropriate to provide the LLM the class as it's already written and describe the new functionality you're trying to add.
* **Provide Feedback:** Iteratively prompt the LLM and provide feedback on the output you receive to get closer to your expected results. In this case, you could try the code you develop alongside the LLM and report back on bugs, unexpected behavior, or stylistic decisions you want improved.
* **Assign a Role:** Assign a role to tailor the output you receive from the LLM. At first you might just want to assign the role of "an experienced Python developer" but later on try out more specific or expert roles to focus on areas like security or scalability. 

**Testing Your Class:** At the bottom of this notebook you'll find different test cases that will help determine if your class works as expected. This lab is ungraded, however, so you don't need to pass all the test cases to move on. Focus instead on exploring what coding alongside an LLM is like, trying the prompting skills, and building your own intuitive sense of how LLMs will best fit into your software development workflow.

<a name="1.1"></a>
### 1.1 Importing necessary libraries

In [8]:
import threading # Used to make the class thread-safe

<a name="2"></a>
## 2 - The `Node` and `LinkedList` Classes to Update
Below are the classes you saw in the lectures and that you will be editing. Recall that a linked list is made up of individual nodes that have connections between one another. This class initially is a singly linked list, meaning each node only knows the location of the node that comes after it in the linked list. In a doubly linked list the nodes also know the location of the node that comes before it. 

**Update both these classes to make the linked list doubly-linked.**

In [9]:
class Node:
    """
    A class representing a node in a doubly linked list.

    Attributes:
    ----------
    data : any
        The data stored in the node.
    next : Node, optional
        The reference to the next node in the list (default is None).
    prev : Node, optional
        The reference to the previous node in the list (default is None).
    """
    def __init__(self, data):
        """
        Initializes a new node with the given data.

        Parameters:
        ----------
        data : any
            The data to be stored in the node.
        """
        self.data = data
        self.next = None
        self.prev = None  # New attribute to store the reference to the previous node

class LinkedList:
    """
    A class representing a doubly linked list.

    Attributes:
    ----------
    head : Node, optional
        The reference to the first node in the list (default is None).
    tail : Node, optional
        The reference to the last node in the list (default is None).
    size : int
        The current size of the list.
    max_size : int, optional
        The maximum size of the list (default is None, meaning no limit).
    lock : threading.Lock
        A lock to ensure thread safety during list operations.
    max_limit : int
        The upper limit for max_size to prevent uncontrolled growth.
    max_data_size : int
        The maximum allowed size for data in a node.
    """
    def __init__(self, max_size=None, max_limit=10000, max_data_size=1000):
        """
        Initializes a new doubly linked list.

        Parameters:
        ----------
        max_size : int, optional
            The maximum size of the list (default is None, meaning no limit).
        max_limit : int, optional
            The upper limit for max_size to prevent uncontrolled growth (default is 10000).
        max_data_size : int, optional
            The maximum allowed size for data in a node (default is 1000).
        """
        self.head = None
        self.tail = None  # New attribute to store the reference to the last node
        self.size = 0
        self.max_size = max_size  
        self.max_limit = max_limit  # Upper limit for max_size to prevent uncontrolled growth
        self.max_data_size = max_data_size  # Maximum allowed size for data in a node
        self.lock = threading.Lock()

    def append(self, data):
        """
        Appends a new node with the given data to the end of the list.

        Parameters:
        ----------
        data : any
            The data to be stored in the new node. The data size must not exceed the configured maximum data size.

        Raises:
        ------
        ValueError
            If the data size exceeds the configured maximum data size.
        MemoryError
            If resizing the list exceeds the maximum limit.
        """
        try:
            # Validate input data
            if isinstance(data, (str, list, tuple, dict)):
                if len(data) > self.max_data_size:
                    raise ValueError(f"Data size exceeds maximum limit of {self.max_data_size} characters.")
            elif isinstance(data, (int, float)):
                if len(str(data)) > self.max_data_size:
                    raise ValueError(f"Data size exceeds maximum limit of {self.max_data_size} characters.")
            else:
                raise TypeError("Unsupported data type.")
            
            with self.lock:
                if self.max_size is not None and self.size >= self.max_size:
                    if self.max_size * 2 > self.max_limit:
                        raise MemoryError("Cannot resize beyond the maximum limit.")
                    self.resize(self.max_size * 2)  # Double the max size when the list is full
                new_node = Node(data)
                if self.head is None:
                    self.head = self.tail = new_node
                else:
                    self.tail.next = new_node
                    new_node.prev = self.tail
                    self.tail = new_node
                self.size += 1
        except Exception as e:
            print(f"Error appending data: {e}")

    def resize(self, new_max_size):
        """
        Resizes the maximum size of the list.

        Parameters:
        ----------
        new_max_size : int
            The new maximum size of the list. It must be greater than the current max size and not exceed the maximum limit.

        Raises:
        ------
        ValueError
            If the new max size is not greater than the current max size.
        MemoryError
            If the new max size exceeds the maximum limit.
        """
        try:
            if new_max_size <= self.max_size:
                raise ValueError("New max size must be greater than the current max size.")
            if new_max_size > self.max_limit:
                raise MemoryError("New max size exceeds the maximum limit.")
            self.max_size = new_max_size
        except Exception as e:
            print(f"Error resizing list: {e}")

    def print_list(self):
        """
        Prints the data of all nodes in the list from head to tail.

        Raises:
        ------
        Exception
            If an error occurs while printing the list.
        """
        try:
            current = self.head
            if current is None:
                print("The list is empty.")
                return
            while current:
                print(current.data, end=" ")
                current = current.next
            print()
        except Exception as e:
            print(f"Error printing list: {e}")

    def print_list_reverse(self):
        """
        Prints the data of all nodes in the list from tail to head.

        Raises:
        ------
        Exception
            If an error occurs while printing the list in reverse.
        """
        try:
            current = self.tail
            if current is None:
                print("The list is empty.")
                return
            while current:
                print(current.data, end=" ")
                current = current.prev
            print()
        except Exception as e:
            print(f"Error printing list in reverse: {e}")

# Example usage
if __name__ == "__main__":
    try:
        dll = LinkedList(max_size=5, max_limit=100, max_data_size=500)  # Set a reasonable upper limit and max data size
        dll.append("Node 1")
        dll.append("Node 2")
        dll.append("Node 3")
        dll.append("Node 4")
        dll.append("Node 5")
        dll.append("Node 6")  # This will trigger resizing
        dll.print_list()  # Output: Node 1 Node 2 Node 3 Node 4 Node 5 Node 6
        dll.print_list_reverse()  # Output: Node 6 Node 5 Node 4 Node 3 Node 2 Node 1
    except Exception as e:
        print(f"Error in example usage: {e}")


Node 1 Node 2 Node 3 Node 4 Node 5 Node 6 
Node 6 Node 5 Node 4 Node 3 Node 2 Node 1 


<a name="3"></a>
## 3 - Test Your Classes
Below are three tests that should help you validate that your updated class is working as intended.

In [10]:
# Test 1 - Append Multiple Data Types
# As initially designed not all data types can be added to the linked list.
# Update the code to allow for additional data types.

linked_list = LinkedList()
linked_list.append("A")
linked_list.append(1)
linked_list.append(0.1)
linked_list.print_list()

# Expected Output:
# A 1 0.1

A 1 0.1 


In [11]:
# Test 2 - Print the Linked List in Reverse
# Write the print_list_reverse method. Once your list is doubly linked
# this should be a much easier method to write

linked_list = LinkedList()
linked_list.append("A")
linked_list.append("B")
linked_list.append(10)
linked_list.append(20)
linked_list.print_list_reverse()

# Expected Output:
# 20 10 B A

20 10 B A 


In [12]:
%%timeit
# Test 3 - Append 10,000 items rapidly
# As initially written this is a very slow process. Your updated class
# should be able to find the tail of your linked list (the last node)
# very quickly, significantly speeding up this process.
# Runtimes will vary substantially but as initially written the append method
# will take well more than a second. A refactored doubly linked list class
# should take significantly less than a second.

linked_list = LinkedList()
for i in range(10000):
    linked_list.append("A")

7.66 ms ± 338 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


<a name="4"></a>
## 4 - Go Further with Your LLM Prompting Skills

The three tests above are simple checks that your class is doubly linked, but it's by no means comprehensive of every concern you'd have about the design of this class. Take some time to experiment with either additional functionality you'd want to add, or prompt the LLM to suggest additions based on new roles, like one of a security or scalability expert. Remember, the most important part of this activity is building your skills working with an LLM, so come up with interesting ways to test what these tools are able to help you accomplish.